In [1]:
from math import gcd as bltin_gcd
from random import random, randint
from sympy import isprime

In [2]:
def random_where(lb, ub, where):
    while True:
        n = randint(lb, ub)
        if where(n): return n
        
def random_bbs(n_bits, p=None, q=None, x0=None):
    lb, ub = 2**32, 2**40
    
    # Wygeneruj p i q dające w dzieleniu przez 4 resztę 3
    p = p or random_where(lb, ub, where=lambda x: x % 4 == 3 and isprime(x))
    q = q or random_where(lb, ub, where=lambda x: x % 4 == 3 and isprime(x))
    
    # Oblicz iloczyn
    n = p * q
    
    # Ustaw x0
    x0 = x0 or random_where(0, 2**32, where=lambda x: bltin_gcd(n, x) == 1)
    x0 = x0**2 % n
    assert bltin_gcd(n, x0) == 1
    x = x0
    r = 0
    
    # Wykonuj iteracje gdzie kolejny x = x^2 mod n
    series = []
    for _ in range(n_bits):
        x = pow(x, 2, n)
        r = (r << 1) | (x & 1)
        series.append(x & 1)
         
    assert p % 4 == 3 and q % 4 == 3 and bltin_gcd(n, x) == 1
    print(f'p  = {p}\nq  = {q}\nn  = {n}\nx0 = {x0}')
    print(f'r  = {str(r)[:50]}')
    
    return (r, series)

r,series = random_bbs(n_bits=20000)

p  = 587226446347
q  = 474213727087
n  = 278470841766265106101189
x0 = 562460711186073249
r  = 27097820804802210522137402517723114328809880023284


In [3]:
def bin2dec(arr):
    p = 1
    res = 0
    for f in arr[::-1]:
        res += p * f
        p *= 2

    return res

# Test pojedyńczych bitów
def test1(series):
    L1 = sum(series)
    print("L(1):", L1)
    if 9725 < L1 < 10275:
        print("single bit test passed")
        return True
    else:
        print("single bit test failed")
        return False


# Test serii
def test2(series):
    counter = 0
    intervals = [
        [2315, 2685],
        [1114, 1386],
        [527, 723],
        [240, 384],
        [103, 209],
        [103, 209],
    ]

    series_test_zeros = [0 for _ in range(6)]
    series_test_ones = [0 for _ in range(6)]
    for i in range(len(series) - 2):
        counter += 1
        if series[i + 1] != series[i]:
            if counter <= 5:
                if series[i] == 0:
                    series_test_zeros[counter - 1] += 1
                else:
                    series_test_ones[counter - 1] += 1
                counter = 0
            else:
                if series[i] == 0:
                    series_test_zeros[5] += 1
                else:
                    series_test_ones[5] += 1

                counter = 0
    print("zeros series", series_test_zeros)
    print("ones series:", series_test_ones)
    for i in range(6):
        if not (
            series_test_ones[i] <= intervals[i][1]
            and series_test_ones[i] >= intervals[i][0]
            and series_test_zeros[i] <= intervals[i][1]
            and series_test_zeros[i] >= intervals[i][0]
        ):
            print("series test failed")
            return False

    print("series test passed")
    return True


# Test długiej serii
def test3(series):
    mx = 0
    for i in range(len(series)):
        num = 1
        index = i
        while series[index] == 1 and index <= len(series) - 2:
            index += 1
            num += 1
            if num > mx:
                mx = num
            if num >= 26:
                print("long series test failed")
                return False
    print("longest series: ", mx)
    print("long series test passed")
    return True


# Test pokera
def test4(series):
    segments = [0 for _ in range(16)]
    fours = [series[i : i + 4] for i in range(0, len(series), 4)]
    for four in fours:
        segments[bin2dec(four)] += 1

    x = 16 / 5000 * sum([segment**2 for segment in segments]) - 5000
    print("poker test value:", x)
    if x > 2.16 and x < 46.17:
        print("poker test passed")
    else:
        print("poker test failed")

In [4]:
print("1. -----------")
test1(series)

print("2. -----------")
test2(series)

print("3. -----------")
test3(series)

print("4. -----------")
test4(series)

1. -----------
L(1): 9952
single bit test passed
2. -----------
zeros series [2506, 1306, 569, 310, 154, 173]
ones series: [2525, 1289, 582, 319, 148, 155]
series test passed
3. -----------
longest series:  14
long series test passed
4. -----------
poker test value: 19.539200000000164
poker test passed
